In [1]:
import pandas as pd
import numpy as np
import random
import pickle

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [2]:
# preprocess dataframe
poster_dir = "../data/posters/"
poster_df = pd.read_csv("../data/MovieGenre.csv", encoding = "ISO-8859-1")
poster_df = poster_df.drop_duplicates(subset=['imdbId'], keep="last")
poster_df["Genre"] = poster_df["Genre"].str.split("|")
poster_df["filename"] = poster_df["imdbId"].astype(str) + ".jpg"
poster_df = poster_df.dropna()
poster_df = poster_df[poster_df["Genre"].apply(lambda t: isinstance(t, list))]
poster_df = poster_df.iloc[random.sample(range(0, 30000), 5000)]

In [3]:
# create image data generator
datagen = ImageDataGenerator(rescale=1./255., validation_split=0.25)

train_generator = datagen.flow_from_dataframe(dataframe=poster_df,
                                              directory=poster_dir,
                                              x_col="filename",
                                              y_col="Genre",
                                              subset="training",
                                              batch_size=32,
                                              shuffle=True,
                                              seed=42,
                                              class_mode="categorical",
                                              target_size=(64, 64))


valid_generator=datagen.flow_from_dataframe(dataframe=poster_df,
                                            directory=poster_dir,
                                            x_col="filename",
                                            y_col="Genre",
                                            subset="validation",
                                            batch_size=32,
                                            shuffle=True,
                                            seed=42,
                                            class_mode="categorical",
                                            target_size=(64, 64))

num_classes = len(train_generator.class_indices)
print(train_generator.class_indices)

Found 3486 validated image filenames belonging to 27 classes.
Found 1161 validated image filenames belonging to 27 classes.
{'Action': 0, 'Adult': 1, 'Adventure': 2, 'Animation': 3, 'Biography': 4, 'Comedy': 5, 'Crime': 6, 'Documentary': 7, 'Drama': 8, 'Family': 9, 'Fantasy': 10, 'Film-Noir': 11, 'Game-Show': 12, 'History': 13, 'Horror': 14, 'Music': 15, 'Musical': 16, 'Mystery': 17, 'News': 18, 'Romance': 19, 'Sci-Fi': 20, 'Short': 21, 'Sport': 22, 'Talk-Show': 23, 'Thriller': 24, 'War': 25, 'Western': 26}


C:\Users\lle\AppData\Roaming\Python\Python38\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 353 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
C:\Users\lle\AppData\Roaming\Python\Python38\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 353 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [8]:
# use CNN model for classification
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(64,64,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
          
model.add(Conv2D(64, (3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))

model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

adam = Adam(learning_rate=7e-4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#callback = EarlyStopping(patience=3)
model.compile(optimizer=adam, loss='binary_crossentropy',
              metrics=['accuracy'])

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

history = model.fit(x=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=50,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID)

Epoch 1/50
108/108 [==============================] - 5s 40ms/step - loss: 0.5546 - accuracy: 0.0693 - val_loss: 0.2457 - val_accuracy: 0.2422
Epoch 2/50
108/108 [==============================] - 4s 38ms/step - loss: 0.3052 - accuracy: 0.1915 - val_loss: 0.2289 - val_accuracy: 0.2431
Epoch 3/50
108/108 [==============================] - 4s 38ms/step - loss: 0.2689 - accuracy: 0.2284 - val_loss: 0.2246 - val_accuracy: 0.2431
Epoch 4/50
108/108 [==============================] - 4s 38ms/step - loss: 0.2580 - accuracy: 0.2551 - val_loss: 0.2292 - val_accuracy: 0.2422
Epoch 5/50
108/108 [==============================] - 4s 38ms/step - loss: 0.2490 - accuracy: 0.2509 - val_loss: 0.2228 - val_accuracy: 0.2431
Epoch 6/50
108/108 [==============================] - 4s 38ms/step - loss: 0.2447 - accuracy: 0.2546 - val_loss: 0.2209 - val_accuracy: 0.2422
Epoch 7/50
108/108 [==============================] - 4s 40ms/step - loss: 0.2416 - accuracy: 0.2491 - val_loss: 0.2210 - val_accuracy: 0.2422

In [ ]:
# save the model for deployment
pickle.dump(model, open('poster_predictor.pkl', 'wb+'))

Reference:
1. https://godatadriven.com/blog/keras-multi-label-classification-with-imagedatagenerator/
2. https://github.com/nddave/Movie-Genre-Prediction